## 自定义层 或者 网络 
+ net = Sequential()
    + net.build(input_shape=)
    + net.summary()

In [ ]:
#继承前代码  改变权重和偏执 自定义Dense
class MyDense(layers.Layer):
    def __init__(self, inp_dim, outp_dim):
        super(MyDense, self).__init__()

        self.kernel = self.add_variable('w', [imp_dim, outp_dim])
        self.bias = self.add_variable('b', [outp_dim])

    def call(self, inputs, training=None):
        out = input @ self.kernel + self.bias

        return out

class MyModel(keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.fc1 = MyDense(28 * 28, 256)
        self.fc2 = MyDense(256, 128)
        self.fc3 = MyDense(128, 64)
        self.fc4 = MyDense(64, 32)
        self.fc5 = MyDense(32, 10)

    def call(self, iputs, training=None):
        x = self.fc1(inputs)
        x = tf.nn.relu(x)
        x = self.fc2(x)
        x = tf.nn.relu(x)
        x = self.fc3(x)
        x = tf.nn.relu(x)
        x = self.fc4(x)
        x = tf.nn.relu(x)
        x = self.fc5(x)

        return x


## 模型加载与保存
+ model.save_weights()/model.load_weights()
+ save/load entire model #记录所有信息
+ saved_model #通用，包括Pytorch、其他语言
+ network.compile()
+ network.fit()
+ network.evaluate()

In [ ]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Restore the weights
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss, acc = model.evaluate(test_images, test_labels)
print(f'Restored model, accuracy: {100*acc:5.2f}')

In [ ]:
tf.saved_model.save(m, '/tmp/saved_model/')

imported = tf.saved_model.load(path)
f = imported.signatures['serving_default']
print(f(x=tf.ones([1, 28, 28, 3])))

## CIFAR10实战

In [5]:
import os
import tensorflow as tf
from tensorflow.keras import datasets, layers, optimizers, Sequential, metrics
from tensorflow import keras

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


def preprocess(x, y):
    # [0, 255] --> [-1,1]
    x = 2 * tf.cast(x, dtype=tf.float32) / 255. - 1
    y = tf.cast(y, dtype=tf.int32)

    return x, y


batch_size = 128
# x --> [32,32,3], y --> [10k, 1]
(x, y), (x_val, y_val) = datasets.cifar10.load_data()
y = tf.squeeze(y)  # [10k, 1] --> [10k]
y_val = tf.squeeze(y_val)
y = tf.one_hot(y, depth=10)  # [50k, 10]
y_val = tf.one_hot(y_val, depth=10)  # [10k, 10]
print('datasets:', x.shape, y.shape, x_val.shape, y_val.shape, x.min(),
      x.max())

train_db = tf.data.Dataset.from_tensor_slices((x, y))
train_db = train_db.map(preprocess).shuffle(10000).batch(batch_size)
test_db = tf.data.Dataset.from_tensor_slices((x_val, y_val))
test_db = test_db.map(preprocess).batch(batch_size)

# sample = next(iter(train_db))
# print('batch:', sample[0].shape, sample[1].shape)


class MyDense(layers.Layer):
    # to replace standard layers.Dense()
    def __init__(self, inp_dim, outp_dim):
        super(MyDense, self).__init__()

        self.kernel = self.add_variable('w', [inp_dim, outp_dim])


#         self.bias = self.add_variable('b', [outp_dim])

    def call(self, inputs, training=None):
        x = inputs @ self.kernel
        return x


class MyNetwork(keras.Model):
    def __init__(self):
        super(MyNetwork, self).__init__()
        self.fc1 = MyDense(32 * 32 * 3, 256)
        self.fc2 = MyDense(256, 128)
        self.fc3 = MyDense(128, 64)
        self.fc4 = MyDense(64, 32)
        self.fc5 = MyDense(32, 10)

    def call(self, inputs, training=None):
        """inputs: [b,32,32,32,3]"""
        x = tf.reshape(inputs, [-1, 32 * 32 * 3])
        # [b,32*32*32] --> [b, 256]
        x = self.fc1(x)
        x = tf.nn.relu(x)
        # [b, 256] --> [b,128]
        x = self.fc2(x)
        x = tf.nn.relu(x)
        # [b, 128] --> [b,64]
        x = self.fc3(x)
        x = tf.nn.relu(x)
        # [b, 64] --> [b,32]
        x = self.fc4(x)
        x = tf.nn.relu(x)
        # [b, 32] --> [b,10]
        x = self.fc5(x)

        return x


network = MyNetwork()
network.compile(optimizer=optimizers.Adam(lr=1e-3),
                loss=tf.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
network.fit(train_db, epochs=5 , validation_data=test_db, validation_freq=1)

network.evaluate(test_db)
# network.save_weights('weights.ckpt')
# del network
# print('saved to ckpt/weights.ckpt')

# network = MyNetwork()
# network.compile(optimizer=optimizers.Adam(lr=1e-3),
#                 loss=tf.losses.CategoricalCrossentropy(from_logits=True),
#                 metircs=['accuracy'])
# network.fit(train_db, epochs=5, validation_data=test_db, validation_freq=1)
# network.load_weights('weights.ckpt')
# print('loaded weights from file.')

# network.evaluate(test_db)


datasets: (50000, 32, 32, 3) (50000, 10) (10000, 32, 32, 3) (10000, 10) 0 255
Epoch 1/5
391/391 [==============================] - 11s 29ms/step - loss: 1.7288 - accuracy: 0.3887 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
391/391 [==============================] - 10s 24ms/step - loss: 1.5077 - accuracy: 0.4681 - val_loss: 1.4906 - val_accuracy: 0.4724
Epoch 3/5
391/391 [==============================] - 10s 26ms/step - loss: 1.4056 - accuracy: 0.5032 - val_loss: 1.4596 - val_accuracy: 0.4891
Epoch 4/5
391/391 [==============================] - 10s 25ms/step - loss: 1.3279 - accuracy: 0.5319 - val_loss: 1.4331 - val_accuracy: 0.5024
Epoch 5/5
79/79 [==============================] - 1s 15ms/step - loss: 1.4071 - accuracy: 0.5068


[1.4071481484401076, 0.5068]